In [29]:
from datasets import load_dataset

dataset = load_dataset("../data/datasets/summary_title") # TODO: create real dataset

dataset

Using custom data configuration summary_title-79ba0bdbe5d8ae9f


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/Marvin Kosmider/.cache/huggingface/datasets/csv/summary_title-79ba0bdbe5d8ae9f/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 1
    })
})

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")

In [40]:
def add_prefix(examples):
    examples["text"] = "Write a title: " + examples["text"]
    return examples

prefixed_dataset = dataset.map(add_prefix)

def add_label(examples):
    examples["label"] = tokenizer(examples["title"], padding="max_length", truncation=True)
    return examples

labeled_dataset = dataset.map(add_label)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = labeled_dataset.map(tokenize_function, batched=True)

tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1
    })
    test: Dataset({
        features: ['title', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1
    })
})

In [41]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [11]:
import numpy as np
from transformers import AutoModelForSeq2SeqLM
from transformers import TrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B")
training_args = TrainingArguments(output_dir="test_trainer")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return (predictions == labels).mean().item()

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, title. If text, title are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
C:\Users\Marvin Kosmider\FH Technikum Wien\Studium\Semester 2\Development Project 1\scientificProject\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3


TypeError: must be real number, not dict